In [1]:
import numpy as np
import pandas as pd
import boto3
import csv
import random
import json
import pickle
from botocore.exceptions import BotoCoreError, ClientError

In [ ]:
bucket_name='<bucket-name>'

In [3]:
data=pd.read_csv(r'C:\CodeFiles\2025-Credit-card-fraud-detection-system\fraudTest.csv')


In [4]:
data.drop(['Unnamed: 0','is_fraud'],axis=1).to_csv('TransactionData.csv',index=False)

In [ ]:
# AWS Setup
region_name = 'us-east-1'
stream_name = 'credit-card-transaction-stream'
kinesis_client = boto3.client('kinesis', region_name=region_name)

def safe_float(value):
    try:
        return float(value)
    except (ValueError, TypeError):
        return 0.0

def safe_int(value):
    try:
        return int(value)
    except (ValueError, TypeError):
        return 0

csv_file_path = 'TransactionData.csv'

# Read and Sample Data
with open(csv_file_path, 'r') as file:
    csv_reader = csv.DictReader(file)
    rows = list(csv_reader)
    selected_records = random.sample(rows, 20)

    for row in selected_records:
        try:
            transaction = {
                'trans_num': row.get('trans_num', 'unknown'),
                'trans_date_trans_time': str(row.get('trans_date_trans_time', '')),
                'merchant': row.get('merchant', ''),
                'category': row.get('category', ''),
                'amt': safe_float(row.get('amt')),
                'city': row.get('city', ''),
                'state':row.get('state',''),
                'lat': safe_float(row.get('lat')),
                'long': safe_float(row.get('long')),
                'city_pop': safe_int(row.get('city_pop')),
                'merch_lat': safe_float(row.get('merch_lat')),
                'merch_long': safe_float(row.get('merch_long')),
            }
            
            # Generate partition key safely
            partition_key = str(row.get('trans_num', row.get('merchant', 'default_key')))
            
            response = kinesis_client.put_record(
                StreamName=stream_name,
                Data=json.dumps(transaction),
                PartitionKey=partition_key
            )
            
            print(json.dumps({
                "status": "Sent to Kinesis",
                "transaction_id": row.get('trans_num', 'unknown'),
                "response": response
            }))
        except (BotoCoreError, ClientError) as e:
            print(f"Error sending transaction to Kinesis: {e}")
        except Exception as e:
            print(f"Unexpected error: {e}")

        